In [61]:
import onnx

print(f"opset={onnx.defs.onnx_opset_version()}, IR_VERSION={onnx.IR_VERSION}")

fa = onnx.load("knn_women.onnx")
fc = onnx.load("knn_girls.onnx")
ma = onnx.load("knn_men.onnx")
mc = onnx.load("knn_boys.onnx")

opset=21, IR_VERSION=10


In [62]:
onnx.compose.add_prefix(fa, "fa_", inplace=True)
onnx.compose.add_prefix(fc, "fc_", inplace=True)
onnx.compose.add_prefix(ma, "ma_", inplace=True)
onnx.compose.add_prefix(mc, "mc_", inplace=True)

merged_f = onnx.compose.merge_models(fa, fc, io_map=[])
merged_m = onnx.compose.merge_models(ma, mc, io_map=[])
merged = onnx.compose.merge_models(merged_f, merged_m, io_map=[])

In [63]:
merged.graph.input[0].name = "input"

for node in merged.graph.node:
    if "Sc_Scan" in node.name:
        node.input.remove(node.input[0])
        node.input.append(merged.graph.input[0].name)

merged.graph.input.remove(merged.graph.input[1])
merged.graph.input.remove(merged.graph.input[1])
merged.graph.input.remove(merged.graph.input[1])


# merged = onnx.version_converter.convert_version(merged, 20)
onnx.save_model(merged, "knn_merged.onnx")